In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install pyannote.audio
!pip install torchaudio
!pip install pydub

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-o83tcqlp
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-o83tcqlp
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.7 MB/s eta 0:00:00
   ━━

In [ ]:

# 📁 Imports
from pydub import AudioSegment
from pyannote.audio import Pipeline
import whisper
import torch
import json
import os
from datetime import datetime


token = "XXXXXXXXXXXXXXXXXXXXXXXX"
#Remplacer

# 📍 Paramètres généraux
audio_path = "/content/drive/MyDrive/AudiosToTranscribe/Archive_001_LoganSamawithJME&NewhamGeneralsKissOct5th2010.mp3"  # à remplacer
archive_title = "LoganSamaNewhamGenerals"
mode_test = False


In [ ]:

# 🎧 Conversion en WAV mono 16kHz
audio = AudioSegment.from_mp3(audio_path)
wav = audio.set_channels(1).set_frame_rate(16000)

# 🎯 Mode test : extrait de 1:15 à 1:45 (30s)
if mode_test:
    start_ms = 75 * 1000
    end_ms = 105 * 1000
    clip = wav[start_ms:end_ms]
    wav_path = "test_clip.wav"
else:
    wav_path = "converted.wav"
    clip = wav

clip.export(wav_path, format="wav")
print(f"✅ Fichier audio exporté : {wav_path}")


✅ Fichier audio exporté : test_clip.wav


In [ ]:

# 📌 Transcription avec Whisper
model = whisper.load_model("medium")
result = model.transcribe(wav_path, word_timestamps=True)
segments = result["segments"]
print(f"✅ {len(segments)} segments transcrits")


100%|█████████████████████████████████████| 1.42G/1.42G [00:28<00:00, 53.5MiB/s]


✅ 17 segments transcrits


In [ ]:
# 🧠 Diarisation
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=hf_token)

# 💡 Tu peux modifier ces valeurs selon ton cas
min_speakers = 4
max_speakers = 6

dz = pipeline(
    {"uri": "audio", "audio": wav_path},
    num_speakers=None,  # automatique si min/max pas utilisés
    min_speakers=min_speakers,
    max_speakers=max_speakers
)
print("✅ Diarisation terminée")

# 🎙️ Indexation par temps
speaker_timeline = []
for turn, _, speaker in dz.itertracks(yield_label=True):
    speaker_timeline.append({
        "start": turn.start,
        "end": turn.end,
        "speaker": speaker
    })

config.yaml:   0%|          | 0.00/500 [00:00<?, ?B/s]

DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover


pytorch_model.bin:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/318 [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


...

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/classifier.ckpt' -> '/root/.cache/torch/pyannote/speechbrain/classifier.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["classifier"] = /root/.cache/torch/pyannote/speechbrain/classifier.ckpt
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/label_encoder.txt' -> '/root/.cache/torch/pyannote/speechbrain/label_encoder.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["label_encoder"] = /root/.cache/torch/pyannote/speechbrain/label_encoder.ckpt
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): embedding_model -> /root/.cache/torch/pyannote/speechbrain/embedding_model.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): mean_var_norm_emb -> /root/.cache/torch/pyannote/speechbrain/mean_var_norm_emb.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): classifier -> /root/.cac

✅ Diarisation terminée


In [ ]:

# 🧩 Construction du JSON
final_data = {
    "title": archive_title,
    "instrumentals": [
        {"start": 0.0, "end": 60.0, "title": "PLACEHOLDER 1", "artist": "TODO"},
        {"start": 60.0, "end": 120.0, "title": "PLACEHOLDER 2", "artist": "TODO"},
        {"start": 120.0, "end": 180.0, "title": "PLACEHOLDER 3", "artist": "TODO"}
    ],
    "segments": []
}

def find_speaker(start, end):
    for s in speaker_timeline:
        if s["start"] <= start < s["end"]:
            return s["speaker"]
    return "unknown"

segment_id = 0
for seg in segments:
    words = seg["words"]
    # Suppression de la clé 'seek' si présente
    for word in words:
        word.pop("seek", None)
    # Découpage par phrases naturelles ou max 12 mots
    for i in range(0, len(words), 12):
        chunk = words[i:i+12]
        segment = {
            "id": segment_id,
            "start": chunk[0]["start"],
            "end": chunk[-1]["end"],
            "text": " ".join([w["word"] for w in chunk]),
            "words": chunk,
            "speaker": find_speaker(chunk[0]["start"], chunk[-1]["end"]),
            "instrumental": None
        }
        final_data["segments"].append(segment)
        segment_id += 1

# Export
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"archive_output_{timestamp}.json"
with open(filename, "w") as f:
    json.dump(final_data, f, indent=2)

print(f"✅ Fichier exporté : {filename}")


✅ Fichier exporté : archive_output_20250519_190425.json
